In [57]:
import pandas as pd


In [58]:

df=pd.read_csv('yelp.csv')
df.head(4)

,Unnamed: 0,sentence,label,source
0,0,Wow... Loved this place.,1,yelp
1,1,Crust is not good.,0,yelp
2,2,Not tasty and the texture was just nasty.,0,yelp
3,3,Stopped by during the late May bank holiday of...,1,yelp


In [59]:
x=df['sentence']
y=df['label']

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [61]:
print("shape of X_train: ",X_train.shape)
print("shape of X_test: ",X_test.shape)

print("shape of y_train: ",y_train.shape)
print("shape of y_test: ",y_test.shape)

shape of X_train:  (800,)
shape of X_test:  (200,)
shape of y_train:  (800,)
shape of y_test:  (200,)


In [62]:
print(X_train[6]) 
print(y_train[6])

Honeslty it didn't taste THAT fresh.)
0


In [63]:
# we will use the Tokenizer class : create a word-to-index dictionary. In the word-to-index dictionary,
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000,lower=True)
tokenizer.fit_on_texts(X_train)



X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1


In [64]:
print(X_train[6]) 
print(y_train[6])

[778, 1, 779, 780, 7, 5, 219, 25, 7, 12, 6, 23, 781, 15, 490, 491]
0


In [65]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100  #We set the maximum size of each list to 100.


X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[3, :])  #the resulting feature vector contains mostly zeros, since you have a fairly short sentence.

[ 40   1 123 775 165  33 489 295  74   6  88  31   8  13  59   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [66]:
from keras.utils import to_categorical
num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_train[0])
#print(len(y_train[0]))

(800, 2)
[1. 0.]


In [67]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,SimpleRNN
from keras import optimizers
from keras.preprocessing.sequence import pad_sequences
import numpy as np
#import numpy as np

#(samples, timesteps, features)
#(batch_size, timesteps, input_dim)
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)  #(750, 100, 1)

X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

(800, 100, 1)
(200, 100, 1)


In [68]:

num_classes=2

def vanilla_rnn():
    model = Sequential()
    #model.add(SimpleRNN(50, input_shape = (X_train.shape[1], X_train.shape[2],), return_sequences = False))
    model.add(SimpleRNN(50, input_shape = (maxlen,1), return_sequences = False))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.summary()
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [69]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn = vanilla_rnn, epochs = 5, batch_size = 50)
model.fit(X_train, y_train)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 50)                2600      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 102       
_________________________________________________________________
activation_4 (Activation)    (None, 2)                 0         
Total params: 2,702
Trainable params: 2,702
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
800/800 [==============================] - 1s 1ms/step - loss: 0.7038 - accuracy: 0.5238
Epoch 2/5
800/800 [==============================] - 1s 743us/step - loss: 0.6925 - accuracy: 0.5288
Epoch 3/5
800/800 [==============================] - 1s 773us/step - loss: 0.6927 - accuracy: 0.5263
Epoch 4/5
800/800 [==============================] - 1s 723us/step - loss: 0.6954

In [70]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

print(accuracy_score(y_pred, y_test_))

0.485


# Testing model on New input

In [73]:

a= ['Waitress was bad in service.']
#a = ["I love their service"]
a = tokenizer.texts_to_sequences(a)
a=np.array(a)
a = pad_sequences(a, padding='post', maxlen=maxlen)

a = a.reshape((a.shape[0], a.shape[1], 1))
print(a.shape)

prediction = model.predict(np.array(a))
print(prediction) # 0 means negative and 1 means positive


(1, 100, 1)
[1]
